In [13]:
import yfinance as yf
import kmapper as km
from kmapper.jupyter import display
from umap import UMAP
import sklearn
import sklearn.manifold as manifold
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# read text file with ticker names
filename = open("SP500_tickernames.txt", "r")
raw_tickernames = filename.read()
ticker_names = raw_tickernames.split("\n")
ticker_names = ticker_names[:len(ticker_names)-1]

In [3]:
# define date range
start_date_string = "2020-01-01"
end_date_string = "2022-04-02"

# pull historical data
raw_data = yf.download(ticker_names, start=start_date_string, end=end_date_string)

[*********************100%***********************]  495 of 495 completed

31 Failed downloads:
['NLOK', 'DISCA', 'NLSN', 'DISH', 'PEAK', 'FLT', 'PBCT', 'BLL', 'ABC', 'FBHS', 'DRE', 'WRK', 'ABMD', 'CTXS', 'DISCK', 'RE', 'FRC', 'CDAY', 'CERN', 'XLNX', 'VIAC', 'ANTM', 'FB', 'ATVI', 'PXD', 'INFO', 'PKI', 'TWTR']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
['SIVB']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2020-01-01 -> 2022-04-02)')
['FISV', 'SBNY']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2020-01-01 -> 2022-04-02) (Yahoo error = "Data doesn\'t exist for startDate = 1577854800, endDate = 1648872000")')


In [4]:
# get daily close prices and drop missing columns
df_close = raw_data['Adj Close'].dropna(axis='columns')

In [17]:
df_close

Ticker,A,AAL,AAP,AAPL,ABBV,ABT,ACN,ADBE,ADI,ADM,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-02 00:00:00+00:00,83.347336,28.982893,145.572113,72.876099,72.841980,80.065521,196.058395,334.429993,110.377762,40.693859,...,140.138123,54.000290,56.088295,53.747150,75.451195,93.541389,139.936554,259.140015,44.369530,129.515060
2020-01-03 00:00:00+00:00,82.009140,27.548195,145.581223,72.167603,72.150551,79.089455,195.731873,331.809998,108.434708,40.614433,...,138.059448,54.259995,55.637379,53.152374,75.895287,93.248428,139.570770,256.049988,43.743416,129.534378
2020-01-06 00:00:00+00:00,82.251556,27.219410,143.188705,72.742661,72.719933,79.503830,194.453705,333.709991,107.160721,40.296711,...,137.786179,54.182083,56.064564,53.445045,75.403931,93.193504,138.764175,258.010010,43.210365,128.539902
2020-01-07 00:00:00+00:00,82.503700,27.119778,141.490158,72.400543,72.305099,79.061829,190.255493,333.390015,109.598717,39.811317,...,138.440063,54.069542,55.605736,53.728275,75.120468,93.358299,138.642242,256.470001,42.948082,128.974365
2020-01-08 00:00:00+00:00,83.318245,27.737495,139.864700,73.565201,72.817543,79.384109,190.628647,337.869995,110.588554,39.370045,...,139.308594,54.017597,54.767193,54.294727,75.375587,93.523094,140.302307,247.639999,43.371128,128.694397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-28 00:00:00+00:00,133.100739,17.299999,198.160248,173.180328,147.224899,114.368202,318.213684,450.010010,159.844360,86.758888,...,78.854431,65.525688,75.924477,47.963993,84.222054,115.519585,123.411331,422.279999,63.113716,185.198517
2022-03-29 00:00:00+00:00,136.049316,18.160000,202.829651,176.494049,147.415756,114.558838,327.786896,466.329987,162.634933,82.521919,...,80.448639,66.068367,75.521065,47.858128,86.568840,116.501389,127.240952,438.100006,63.122749,188.044388
2022-03-30 00:00:00+00:00,133.140060,18.049999,199.412308,175.320450,148.842850,114.739922,325.641327,460.059998,159.098969,83.893547,...,79.877869,66.519073,76.813820,47.684910,85.434219,115.185966,126.367012,429.609985,60.977638,187.105560


In [5]:
# convert pandas dataframe to numpy array, standardize ticker data, and transpose array
data = df_close.to_numpy()
data = data-np.mean(data, axis=0)/np.std(data, axis=0)
data = data.transpose()

In [6]:
# calculate percent return of each ticker over date range
per_return = (df_close.to_numpy().transpose()[:,504] - df_close.to_numpy().transpose()[:,0])/df_close.to_numpy().transpose()[:,0]

In [7]:
# initialize mapper
mapper = km.KeplerMapper(verbose=1)

KeplerMapper(verbose=1)


In [8]:
# project data into 2D subsapce via 2 step transformation, 1)isomap 2)UMAP
projected_data = mapper.fit_transform(data, projection=[manifold.Isomap(n_components=100, n_jobs=-1), UMAP(n_components=2,random_state=1)])

..Composing projection pipeline of length 2:
	Projections: Isomap(n_components=100, n_jobs=-1)
		UMAP(random_state=1)
	Distance matrices: False
False
	Scalers: MinMaxScaler()
MinMaxScaler()
..Projecting on data shaped (464, 568)

..Projecting data using: 
	Isomap(n_components=100, n_jobs=-1)



/Users/charlesmiller/.local/share/virtualenvs/dynamics_lab-5gulJLF-/lib/python3.9/site-packages/sklearn/manifold/_isomap.py:383: UserWarning: The number of connected components of the neighbors graph is 2 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
/Users/charlesmiller/.local/share/virtualenvs/dynamics_lab-5gulJLF-/lib/python3.9/site-packages/scipy/sparse/_index.py:108: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])



..Scaling with: MinMaxScaler()

..Projecting on data shaped (464, 100)

..Projecting data using: 
	UMAP(random_state=1, verbose=1)

UMAP(n_jobs=1, random_state=1, verbose=1)


/Users/charlesmiller/.local/share/virtualenvs/dynamics_lab-5gulJLF-/lib/python3.9/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


Mon Sep 23 14:17:54 2024 Construct fuzzy simplicial set
Mon Sep 23 14:17:54 2024 Finding Nearest Neighbors
Mon Sep 23 14:17:55 2024 Finished Nearest Neighbor Search
Mon Sep 23 14:17:56 2024 Construct embedding


Epochs completed:  25%| ██▍        124/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs


Epochs completed: 100%| ██████████ 500/500 [00:00]

	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Mon Sep 23 14:17:57 2024 Finished embedding

..Scaling with: MinMaxScaler()



In [9]:
# cluster data using DBSCAN
G = mapper.map(projected_data, data, clusterer=sklearn.cluster.DBSCAN(metric="cosine"))

Mapping on data shaped (464, 568) using lens shaped (464, 2)

Creating 100 hypercubes.

Created 59 edges and 46 nodes in 0:00:00.053290.


In [11]:
fileID = "test"

In [12]:
# visualize graph
mapper.visualize(G, 
                path_html="mapper_example_" + fileID + ".html",
                title=fileID,
                custom_tooltips = df_close.columns.to_numpy(),
                color_values = np.log(per_return+1),
                color_function_name = 'Log Percent Returns',
                node_color_function = np.array(['average', 'std', 'sum', 'max', 'min']))

# display mapper in jupyter
km.jupyter.display("mapper_example_" + fileID + ".html")

Wrote visualization to: mapper_example_test.html


/Users/charlesmiller/.local/share/virtualenvs/dynamics_lab-5gulJLF-/lib/python3.9/site-packages/kmapper/visuals.py:344: RuntimeWarning: invalid value encountered in scalar divide
  height = np.floor(((bar / max_bucket_value) * 100) + 0.5)
/Users/charlesmiller/.local/share/virtualenvs/dynamics_lab-5gulJLF-/lib/python3.9/site-packages/kmapper/visuals.py:345: RuntimeWarning: invalid value encountered in scalar divide
  perc = round((bar / sum_bucket_value) * 100.0, 1)


/Users/charlesmiller/.local/share/virtualenvs/dynamics_lab-5gulJLF-/lib/python3.9/site-packages/IPython/core/display.py:431: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")
